# Trajectory estimation

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import os

%matplotlib inline
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

## Read and plot dataset

In [ ]:
dimension = 2 # can be 2 or 3.
n_measurements = -1 # set to None or -1 to use all. 
# chosen_distance = 'distance_tango_2D'
chosen_distance = 'distance'
chosen_anchor_names = ['RTT 0', 'RTT 1', 'RTT 2', 'RTT 6']

sigma = 10 # noise for simulated measurements

out_dir = 'experiments/robot_test/'
anchorsfile = 'experiments/anchors.csv'

# All comments are with Tango, right?
name = 'circle2_double_0' # OK, the speed 1/7
# name = 'circle2_double_1' # OK, the speed  1/7 
# (is it possible that we get more correct resutl with 1/6 but the Tango is off?)
# name = 'circle3_triple_0' # OK, the speed  1/6
# name = 'circle3_triple_2' # OK, the speed  1/6
# name = 'clover_0' # does not work, obviously
# name = 'straight1_0' # good
# name = 'straight2_0' # good
# name = 'straight3_0' # good now
# name = 'straight4_0' # good now
# name = 'straight5_0' # good now
# name = 'straight6_0' # good

df = pd.read_pickle(out_dir + name + '_calibrated.pkl')
df.reset_index(inplace=True, drop=True)
df.timestamp -= df.timestamp.values[0]
df.head()

In [ ]:
tango_df = df[df.system_id=='Tango']
plt.figure()
sns.scatterplot(x='px', y='py', hue='timestamp', data=tango_df)
plt.title('2D of Tango data')
plt.xlabel('x [m]'); plt.ylabel('y [m]')
plt.axis('equal')
plt.figure()
plt.plot(tango_df.timestamp, tango_df.pz)
plt.title('z of Tango data')
plt.xlabel('time [s]'); plt.ylabel('z [m]')

In [ ]:
from evaluate_dataset import read_anchors_df
anchors_df = read_anchors_df(anchorsfile)

all_anchor_names = set(sorted(df[df.system_id=='RTT'].anchor_name.unique()))
anchor_names = list(sorted(all_anchor_names.intersection(chosen_anchor_names)))
n_anchors = len(anchor_names)

# if we do not have all we could do something elegant with pandas. 
anchors = anchors_df.loc[anchors_df.anchor_name.isin(anchor_names), ['px', 'py', 'pz']].values.T

for anchor_name in anchor_names:
    anchor_coord = anchors_df.loc[anchors_df.anchor_name==anchor_name, ['px','py']].values
    points = tango_df.loc[:, ['px', 'py']].values
    vecs = points.reshape((-1, 2)) - anchor_coord.reshape((1, 2))
    distances = np.linalg.norm(vecs.astype(np.float32), axis=1)

    df_rtt0 = df[df.anchor_name == anchor_name]

    plt.figure()
    plt.plot(tango_df.timestamp, distances, label='tango')
    for distance in ['distance_tango_2D', 'distance_tango', 'distance', 'distance_median_all', 'distance_median_0']:
        plt.scatter(df_rtt0.timestamp, df_rtt0.loc[:, distance], s=2, label=distance)
    plt.xlabel('time [s]'); plt.ylabel('distance [m]'); plt.legend()
    plt.title(anchor_name)

## Construct D, basis and anchors

In [ ]:
from global_variables import TAU, ROBOT_HEIGHT
from evaluate_dataset import get_length

# EITHER: compute distances from ground truth
lengths = get_length(tango_df)
distances = np.cumsum(lengths)

# OR: compute distances from constant speed assumption.
times = df.iloc[:n_measurements].timestamp.unique().astype(np.float32)
speed = 1/6. # should be 10m/min
distances = speed * times #* TAU # TODO need to understand why TAU.

plt.figure()
plt.hist(distances[times>0] / times[times>0])
plt.title('speed histogram')

print('used speed:', speed)
print('average speed:',np.mean(distances[times>0] / times[times>0]))

In [ ]:
# get times in terms of trajectory
from trajectory_creator import get_trajectory
#from trajectory import Trajectory
traj = get_trajectory(name)
times_corr, _, _  = traj.get_times_from_distances(arbitrary_distances=distances, plot=False, time_steps=10000)
basis = traj.get_basis(times=times_corr)

fig, ax = plt.subplots()
ax = traj.plot(times=times_corr, alpha=0.5)

## real measurements

In [ ]:
from evaluate_dataset import compute_distance_matrix

D_topright_real = compute_distance_matrix(df, anchors_df, anchor_names, times, 
                                          chosen_distance, dimension=dimension, 
                                          robot_height=ROBOT_HEIGHT)
        
plt.figure()
plt.matshow(D_topright_real[:40, :].T)
plt.title('first 40 rows of D_topright, transposed')

## simulated measurements

In [ ]:
# now we need the real trajectory, so we do not scramble.
traj = get_trajectory(name)

points = traj.get_sampling_points(basis=basis)
plt.figure()
plt.scatter(*points[:2])
plt.axis('equal')

from measurements import get_D_topright, add_noise
D_topright_sim = get_D_topright(anchors=anchors[:dimension], samples=points[:2])
np.random.seed(1)
D_topright_sim = add_noise(D_topright_sim, sigma, noise_to_square=False)

## Estimate trajectory
### Using Tango measurements with added noise

In [ ]:
from solvers import alternativePseudoInverse

#D_topright = D_topright_sim
# D_topright = D_topright_real
if 'tango' in chosen_distance:
    for sigma in [0., 0.1, 1, 2, 3, 4]:
        D_topright = add_noise(D_topright_real, sigma, noise_to_square=False)
        print(anchors.shape)

        coeffs = alternativePseudoInverse(D_topright, anchors[:2], basis, 
                                          weighted=False)
        print(coeffs)
        #print(traj.coeffs)
        trajectory_estimated = traj.copy()
        trajectory_estimated.set_coeffs(coeffs=coeffs)
        plt.figure()
        ax = trajectory_estimated.plot()
        points = trajectory_estimated.get_sampling_points(basis=basis)
        points_true = traj.get_sampling_points(basis=basis)
        points_tango = tango_df.loc[:, ['px', 'py']].values.T
        ax.scatter(*points_true[:2], label='model')
        ax.scatter(*points[:2], label='reconstructed')
        ax.plot(*points_tango[:2], label='tango', color='black')
        for a_coord, a_name in zip(anchors[:2].T, anchor_names):
            ax.scatter(*a_coord, color='black'); ax.annotate(s=a_name, xy=a_coord, color='black')
        plt.axis('equal')
        if dimension > 2:
            plt.figure()
            plt.plot(times, points_true[2], label='model')
            plt.plot(times, points[2], label='reconstructed')
        plt.title(r"$\sigma$: {}".format(sigma))
        plt.legend()

When setting sigma to 10 in simulated measurements, then the recovered trajectory looks very similar to the reconstruction using real noisy measurements. We should investigate why. Hypothesis: we take samples at the same time so maybe we oversample in one region compared to the other region.  

### Plot variance in the RTT data.
If we split measurements from RTTs to different sets by dividing them modulo number of stes (`split`) we get very similar reconstruction. This might mean that there is the bias in our data, but it is also possible that it is an artefact of resampling.

In [ ]:
#D_topright = D_topright_sim
# D_topright = D_topright_real


#Recalibrate using params from the other notebook, for 2D data (yes, I know it's ugly):
[a, b] = [1.1960015109177586, 3.078068231948606]
D_topright_all = (D_topright_real - b)/a

if 'tango' not in chosen_distance:
    split = 20
    fig, ax = plt.subplots(1, figsize=(10, 10))
    points_tango = tango_df.loc[:, ['px', 'py']].values.T
    ax.plot(*points_tango[:2], label='tango', color='black')
    for idx in range(split):
        D_topright = D_topright_all[idx::split]
        coeffs = alternativePseudoInverse(D_topright, anchors[:2], basis[:, idx::split], 
                                          weighted=False)
        trajectory_estimated = traj.copy()
        trajectory_estimated.set_coeffs(coeffs=coeffs)
        points = trajectory_estimated.get_sampling_points(basis=basis)
        points_true = traj.get_sampling_points(basis=basis)
        
        ax.scatter(*points[:2], label='reconstructed', alpha=0.3)
        for a_coord, a_name in zip(anchors[:2].T, anchor_names):
            ax.scatter(*a_coord, color='black'); ax.annotate(s=a_name, xy=a_coord, color='black')
        plt.axis('equal')
        if dimension > 2:
            plt.figure()
            plt.plot(times, points_true[2], label='model')
            plt.plot(times, points[2], label='reconstructed')
        plt.title(r"$\sigma$: {}".format(sigma))
        plt.legend()
    plt.show()

### Plot reconstruction error depending both on noise on distances and wrong speed estimation

In [ ]:
from solvers import alternativePseudoInverse
from measurements import get_D_topright, add_noise

# now we need the real trajectory, so we do not scramble.
generator_name = '_'.join(name.split('_')[:-1])
traj = get_trajectory(generator_name + '.csv')
times = traj.get_times(n_samples=50)
basis = traj.get_basis(times=times)
points = traj.get_sampling_points(basis=basis)
D_topright = get_D_topright(anchors=anchors[:dimension], samples=points[:2])

sigmas = [0.1, 1, 2]
speeds = [0.8, 0.9, 1, 1.1, 1.25]

fig, axis = plt.subplots(len(speeds), len(sigmas), figsize=(15,20))

for j, speed in enumerate(speeds):
    for i, sigma in enumerate(sigmas):
        # Plots look very different for different seeds
        # I fix seed so it is easier to observe the efects of noise and speed
        np.random.seed(3)
        D_topright_noise = add_noise(D_topright, sigma, noise_to_square=False)
        jittered_times = times * speed
        jittered_basis = traj.get_basis(times=jittered_times)
        coeffs = alternativePseudoInverse(D_topright_noise, anchors[:2], jittered_basis, 
                                          weighted=False)
        trajectory_estimated = traj.copy()
        trajectory_estimated.set_coeffs(coeffs=coeffs)
        traj.plot(ax=axis[j][i], c="C0")
        trajectory_estimated.plot(ax=axis[j][i], c="C1")
        points = trajectory_estimated.get_sampling_points(basis=jittered_basis)
        points_true = traj.get_sampling_points(basis=basis)
        axis[j][i].scatter(*points_true[:2], label='model', c="C0")
        axis[j][i].scatter(*points[:2], label='reconstructed', c="C1")

        for a_coord, a_name in zip(anchors[:2].T, anchor_names):
            axis[j][i].scatter(*a_coord, color='black'); ax.annotate(s=a_name, xy=a_coord, color='black')
        axis[j][i].axis('equal')
        axis[j][i].legend()
        axis[j][i].set_title(r"$\sigma$: {}, speed: {:.2f}".format(sigma, speed))

### Plot reconstruction error depending both on noise on distances and noisy times 
This should mimic for example the non constant speed, it's just an approximation, because in practice time noise will be corelated and not iid gaussian like here. But it seems that small amount of jitter is not an issue.

And sorry for copypasting code, but I don't know how much of it we want to keep.

In [ ]:
from solvers import alternativePseudoInverse
from measurements import get_D_topright, add_noise

# now we need the real trajectory, so we do not scramble.
generator_name = '_'.join(name.split('_')[:-1])
traj = get_trajectory(generator_name + '.csv')
times = traj.get_times(n_samples=50)
basis = traj.get_basis(times=times)
points = traj.get_sampling_points(basis=basis)
D_topright = get_D_topright(anchors=anchors[:dimension], samples=points[:2])

sigmas = [0.1, 1, 2]
time_sigmas = [0., 0.05, 0.1, 0.5, 1]

fig, axis = plt.subplots(len(speeds), len(sigmas), figsize=(15,20))

for j, time_sigma in enumerate(time_sigmas):
    for i, sigma in enumerate(sigmas):
        # Plots look very different for different seeds
        # I fix seed so it is easier to observe the efects of noise and speed
        np.random.seed(3)
        D_topright_noise = add_noise(D_topright, sigma, noise_to_square=False)
        jittered_times = times + time_sigma * np.random.randn(len(times))
        jittered_basis = traj.get_basis(times=jittered_times)
        coeffs = alternativePseudoInverse(D_topright_noise, anchors[:2], jittered_basis, weighted=False)
        trajectory_estimated = traj.copy()
        trajectory_estimated.set_coeffs(coeffs=coeffs)
        traj.plot(ax=axis[j][i], c="C0")
        trajectory_estimated.plot(ax=axis[j][i], c="C1")
        points = trajectory_estimated.get_sampling_points(basis=jittered_basis)
        points_true = traj.get_sampling_points(basis=basis)
        axis[j][i].scatter(*points_true[:2], label='model', c="C0")
        axis[j][i].scatter(*points[:2], label='reconstructed', c="C1")

        for a_coord, a_name in zip(anchors[:2].T, anchor_names):
            axis[j][i].scatter(*a_coord, color='black'); ax.annotate(s=a_name, xy=a_coord, color='black')
        axis[j][i].axis('equal')
        axis[j][i].legend()
        axis[j][i].set_title(r"$\sigma$: {}, $\sigma_t$: {}".format(sigma, time_sigma))

## Debugging

In [ ]:
distances = df.loc[df.anchor_name=='RTT 2', chosen_distance].values
print(distances)

# first point
start_point = tango_df.iloc[0][['px', 'py']].values
end_point = tango_df.iloc[-1][['px', 'py']].values

print(start_point, end_point)

#start_time = df.iloc[0].timestamp
#end_time = df.iloc[-1].timestamp
start_time = times_corr[0]
end_time = times_corr[-1]
print(start_time, end_time)
points = trajectory_estimated.get_sampling_points(times=[start_time, end_time]).T.tolist()
print(points)

# anchors is of shape  dimxK
for i in range(anchors.shape[1]):
    a = anchors[:2, i]
    anchor_name = 'RTT {}'.format(i)
    print(anchor_name)
    
    print('start distance')
    print(np.linalg.norm(a - start_point))
    
    dist = df[df.anchor_name==anchor_name].iloc[0].loc[chosen_distance]
    print(dist)
    
    print('end distance')
    print(np.linalg.norm(a - end_point))
    time = df[df.anchor_name==anchor_name].iloc[-1].timestamp
    dist = df[df.anchor_name==anchor_name].iloc[-1].loc[chosen_distance]
    print(dist)